## 로지스틱 회귀 (분류알고리즘)
- 로지스틱 = 로짓이라고 부름
- Logisstic Regression (Logit Regression)은 샘플이 특정 클래스에 속할 확률을 추정하는데 널리 사용됨. 
- 계산법은 입력 특성들의 가중치 합을 계산하고, 편향을 더함 => **선형회귀 처럼 결과값을 바로 출력하지는 않고, 결괏값의 로지스틱 값을 출력함.**
- 로지스틱이란 즉 시그모이드 함수(sigmoid function) **최종적으론 시그모이드 값에 대입되서 출력됨.**

![logistic1](./LogisticRegression_imgs/logistic1.PNG)

![logistic2](./LogisticRegression_imgs/logistic2.jpg)

- **클래스 1에 속한 샘플을 정확히 예측하면 비용이 0에 가까워짐. (왼쪽상단에서 오른쪽 하단으로)**
- **클래스 0에 속한 샘플을 y=0으로 정확히 예측하면 비용이 0에 가까워짐 (오른쪽상단에서 왼쪽 하단으로)**

## sklearn.linear_model.LogisticRegression
- penalty : l1, l2, elasticnet, 'none' / default = 'l2'
- solver = 'warn' : 항상 지정 필요
- C=1.0 : 규제항 값 - 규제값이 커지면 가중치도 커짐.
- multi_class='warn' : 이진분류 or 다중분류 , 기본적으로는 auto (자동설정인것)
- max_iter = 100
- random_state = None

- dual=False 
- fit_intercept = True, 
- intercept_scailing = 1
- n_jobs = None, 
- tol=0.0001
- verbose = 0,
- warn_start= False

### ** penalty - solver **
- 'newton-cg', 'sag' and 'lbfgs' solvers support only l2 penalties.
- 'elasticnet' is only supported by the 'saga' solver. 
- 'l1 penalty' with SAGA solver (allowing ‘multinomial’ + L1)

### predict method
- predict() : 클레스 레이블을 반환하는 메서드 ( 0보다 클 때를 양성 클래스로 판단하여 결과 반환)
- predict_probat() : 클래스에 속할 확률을 반환 (시그모이드 함수 적용하여 계산)

### 규제를 이용한 과대적합 피하기
- 과대적합 : 분산이 크다 / 모델 파라미터가 너무 많아 주어진 데이터에서 너무 복잡한 모델을 만든다.
- 과소적합 : 높은편향, / 
- **분산에 대한 해석 : 분산은 모델을 여러번 훈련했을 때 특정 샘플에 대한 예측의 일관성(변동성)을 측정한다. 모델이 훈련 데이터의 무작위성에 민감하다는 것**
- **편향에 대한 해석 : 편향은 다른 훈련 데이터셋에서 여러번 훈련했을 때 예측이 정확한 값에서 얼마나 벗어났는지 측정, 편향은 무적위성보다 구조적인 에러로 해석**
- 좋은 편향-분산 트레이드 오프를 찾는 방법은 규제를 사용하여 모델의 복잡도를 조정하는 것 
- 규제는 공선성을 다루거나 데이터에서 잡음을 제거하여 과대적합을 방지할 수 있는 매우 유용한 방법
- 과도한 파라미터 값을 제한하기 위해 추가적인 정보(편향)을 주입하는 개념 (default L2 규제)
- **규제가 잘 동작하려면 모든 특성이 비슷한 스케일을 가져야 한다**

In [1]:
# (1) 데이터 로드
from sklearn import datasets
import numpy as np
iris=datasets.load_iris()
X = iris.data[:,[2,3]]
y = iris.target
print(X.shape, y.shape) # (150,2), (150,)

(150, 2) (150,)


In [2]:
# (2) train, test split
from sklearn.model_selection import train_test_split

# 계층화 기능 사용 하여서 클래스 레이블 비율 동일하게 하는 것.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y) 
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape, '\n') # 105, 45 비율로

# (3) 데이터 표준화 진행
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)
print(X_train[:3])
print(X_train_std[:3], '\n')
# (4) 두 배열 합치기 (np.hstack - 두 배열을 왼쪽에서 오른쪽으로 붙이기 / 두 배열을 위에서 아래로 붙이기)
X_combined_std = np.vstack((X_train_std, X_test_std))
y_combined = np.hstack((y_train, y_test))
print(X_combined_std[:3]) # 150, 2
print(y_combined[:3])

(105, 2) (45, 2) (105,) (45,) 

[[1.4 0.2]
 [1.7 0.2]
 [5.3 2.3]]
[[-1.33269725 -1.30728421]
 [-1.16537974 -1.30728421]
 [ 0.84243039  1.44587881]] 

[[-1.33269725 -1.30728421]
 [-1.16537974 -1.30728421]
 [ 0.84243039  1.44587881]]
[0 0 2]


In [5]:
from sklearn.linear_model import LogisticRegression

#- 'newton-cg', 'sag' and 'lbfgs' solvers support only l2 penalties.
#- 'elasticnet' is only supported by the 'saga' solver. 
#- 'l1 penalty' with SAGA solver (allowing ‘multinomial’ + L1)

lr = LogisticRegression(penalty = 'l2',#solver = 'liblinear', # 
                        multi_class='auto', # 이진분류이거나 solver가 liblinear일 때 ovr로 그 외에는 multinomial
                        C=100.0,
                        random_state=77 )
# logistic regression 학습
lr.fit(X_train_std, y_train)

# predict_proba : 훈련샘플이 어떤 클래스에 속할 확률 predict_proba
db = lr.predict_proba(X_test_std)
print(lr.predict_proba(X_test_std[:3]))
print(lr.predict_proba(X_test_std[:3]).argmax(axis=1), '\n')

# predict : 훈련샘플이 어떤 클래스에 속한다는 인덱스
print(lr.predict(X_test_std[:3]), '\n')

# 내가 테스트 하고 싶은 데이터가 있다면 2차원 형태로 바꿔줌.
print(X_test_std[-1].reshape(1,-1), '\n')
print(lr.predict(X_test_std[-1].reshape(1,-1)), '\n')


[[1.52213484e-12 3.85303417e-04 9.99614697e-01]
 [9.93560717e-01 6.43928295e-03 1.14112016e-15]
 [9.98655228e-01 1.34477208e-03 1.76178271e-17]]
[2 0 0] 

[2 0 0] 

[[-1.27692475 -1.43838721]] 

[0] 



### linear_model -> 추가 라이브러리 
![linear_model1](./LogisticRegression_imgs/linear_model1.PNG)
![linear_model1](./LogisticRegression_imgs/linear_model2.PNG)
![linear_model1](./LogisticRegression_imgs/linear_model3.PNG)


In [6]:
### 추가 공부 필요 내용 -> Lasso, Ridge, L1, L2, Elastic 차이점 조사 필요